## Data Wrangling

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, fnmatch

In [2]:
## Import Data Sets 

#eCommerceCosmetics
fileList = fnmatch.filter(os.listdir("./RawData/eCommerceCosmetics"),"*.csv")
eCosmoData = pd.read_csv("./RawData/eCommerceCosmetics/" + fileList[0])
for file in fileList[1:]:
    newData = pd.read_csv("./RawData/eCommerceCosmetics/"+file)
    eCosmoData = pd.concat([eCosmoData, newData], ignore_index=False)
    
    
#eCommerceElectronics
eElecData = pd.read_csv("./RawData/eCommerceElectronics/kz.csv")

#IBM Telco
ibmTelcoData = pd.read_csv("./RawData/IBM-TelcoCustomerChurn.csv")

#IBM Marketing Customer Value Data
ibmMarketingData = pd.read_csv("./RawData/IBM-MarketingCustomerValueData.csv")


#Mall Customer
mallData = pd.read_csv("./RawData/Mall_Customers.csv")

#Online Retail 
uciOnlineRetail1 = pd.read_csv("./RawData/Online Retail - Online Retail.csv")

#Online Retail 2
uciOnlineRetail2 = pd.read_csv("./RawData/online_retail_II - Year 2009-2010.csv")

#ula box orders 
ulaboxOrders = pd.read_csv("./RawData/ulabox_orders_with_categories_partials_2017.csv")


In [3]:
#OList eCommerce Public Data Set
olistDir ="./RawData/OlistPublicECommerce/"

OL_customers = pd.read_csv(olistDir+"olist_customers_dataset.csv")
OL_geo = pd.read_csv(olistDir+"olist_geolocation_dataset.csv")
OL_orders = pd.read_csv(olistDir+"olist_orders_dataset.csv")
OL_orders_items = pd.read_csv(olistDir+"olist_order_items_dataset.csv")
OL_orders_payments = pd.read_csv(olistDir+"olist_order_payments_dataset.csv")
OL_orders_reviews = pd.read_csv(olistDir+"olist_order_reviews_dataset.csv")
OL_products = pd.read_csv(olistDir+"olist_products_dataset.csv")
OL_sellers = pd.read_csv(olistDir+"olist_sellers_dataset.csv")
OL_products_translations = pd.read_csv(olistDir+"product_category_name_translation.csv")


OL_customers    = OL_customers.dropna(thresh=len(OL_customers.columns)-1)
OL_geo          = OL_geo.dropna(thresh=len(OL_geo.columns)-1)
OL_orders       = OL_orders.dropna(thresh=len(OL_orders.columns)-1)
OL_orders_items = OL_orders_items.dropna(thresh=len(OL_orders_items.columns)-1)
OL_orders_payments = OL_orders_payments.dropna(thresh=len(OL_orders_payments.columns)-1)
OL_orders_reviews  = OL_orders_reviews.dropna(thresh=len(OL_orders_reviews.columns)-1)
OL_products        = OL_products.dropna(thresh=len(OL_products.columns)-1)
OL_sellers         = OL_sellers.dropna(thresh=len(OL_sellers.columns)-1)
OL_products_translations = OL_products_translations.dropna(thresh=len(OL_products_translations.columns)-1)

<h5>OL Customer Merge</h5>

In [4]:
dropGeo = OL_geo.drop_duplicates("geolocation_zip_code_prefix")
dropGeo = dropGeo[['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng']]

mergedOLCustomers = OL_customers.merge(dropGeo, how='left',left_on="customer_zip_code_prefix", right_on="geolocation_zip_code_prefix")
mergedOLCustomers = mergedOLCustomers.rename(columns={'geolocation_lat':'customer_latitude','geolocation_lng':'customer_longitude'})
mergedOLCustomers = mergedOLCustomers.drop("geolocation_zip_code_prefix",axis=1)

<h5>OL Items Merge</h5>

In [5]:
productsSub = OL_products[['product_id', 'product_category_name', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']]
productsSub.insert(len(productsSub.columns),'product_volume_cm3', OL_products['product_length_cm']*OL_products['product_height_cm']*OL_products['product_width_cm'])
productsSub.insert(len(productsSub.columns),'product_density_g_per_cm3',OL_products['product_weight_g']/productsSub['product_volume_cm3'])

mergedOLProducts = productsSub.merge(OL_products_translations, how='left', on="product_category_name")
mergedOLProduct = mergedOLProducts.drop("product_category_name",axis=1)
mergedOLItems = OL_orders_items.merge(mergedOLProducts, how='left', on="product_id")
mergedOLItems = mergedOLItems.drop("product_category_name",axis=1)

<h5>OL Sellers Merge</h5>

In [6]:
mergedSellers = OL_sellers.merge(dropGeo, how='left', left_on="seller_zip_code_prefix", right_on="geolocation_zip_code_prefix")
mergedSellers = mergedSellers.drop("geolocation_zip_code_prefix",axis=1)
mergedSellers = mergedSellers.rename(columns={'geolocation_lat':'seller_latitude','geolocation_lng':'seller_longitude'})
mergedOLItems = mergedOLItems.merge(mergedSellers, how='left', on="seller_id")

<h5>OL Data Merge</h5>

In [7]:
ReviewsSub = OL_orders_reviews[['review_id', 'order_id', 'review_score', 'review_creation_date', 'review_answer_timestamp']]

OL_Data = OL_orders.merge(OL_orders_payments, how='left', on='order_id')
OL_Data = OL_Data.merge(ReviewsSub, how='left', on='order_id')
OL_Data = OL_Data.merge(mergedOLItems, how='left', on='order_id')
OL_Data = OL_Data.merge(mergedOLCustomers, how='left', on='customer_id')


<h3> Function Definitions</h3>

In [8]:
import itertools
def PrintDataFrameSummary(frame):
    metaData = list()
    metaData.append(list(["Name:","DType:","Non-Null:","Unique:"]))
    for col in list(frame.columns):
        metaData.append( list(["{0}".format(col), 
                              "{0}".format(frame[col].dtype),
                              "{0}".format(frame[col].notna().sum()), 
                              "{0}".format(len(set(frame[frame[col].notna()][col])))]))
    
    formatString = ""
    
    for i in range(len(metaData[0])):
        maxLength = len(max([l[i] for l in metaData], key=len))
        formatString += "{"+":<{0}".format(maxLength+8) +"}"
        
    for i in range(len(metaData)):
        
        print(formatString.format(*metaData[i]))
    
    
        
    print()
    print("Initial Length {0}".format(len(frame)))
    print("Dropped NA Length {0}".format(len(frame.dropna())))
    return frame.head()

<h2>Part 1) Initial investigation</h2>

<div class="span5 alert alert-info">
<h3>Online Cosmetic Store Data</h3>
</div>

In [9]:
PrintDataFrameSummary(eCosmoData)

Name:                DType:         Non-Null:        Unique:        
event_time           object         20692840         8782890        
event_type           object         20692840         4              
product_id           int64          20692840         54571          
category_id          int64          20692840         525            
category_code        object         353594           12             
brand                object         11935723         273            
price                float64        20692840         2860           
user_id              int64          20692840         1639358        
user_session         object         20688242         4535941        

Initial Length 20692840
Dropped NA Length 266709


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-12-01 00:00:00 UTC,remove_from_cart,5712790,1487580005268456287,NaN,f.o.x,6.27,576802932,51d85cb0-897f-48d2-918b-ad63965c12dc
1,2019-12-01 00:00:00 UTC,view,5764655,1487580005411062629,NaN,cnd,29.05,412120092,8adff31e-2051-4894-9758-224bfa8aec18
2,2019-12-01 00:00:02 UTC,cart,4958,1487580009471148064,NaN,runail,1.19,494077766,c99a50e8-2fac-4c4d-89ec-41c05f114554
3,2019-12-01 00:00:05 UTC,view,5848413,1487580007675986893,NaN,freedecor,0.79,348405118,722ffea5-73c0-4924-8e8f-371ff8031af4
4,2019-12-01 00:00:07 UTC,view,5824148,1487580005511725929,NaN,NaN,5.56,576005683,28172809-7e4a-45ce-bab0-5efa90117cd5


<div class="span5 alert alert-info">
    <p><b>Positives:</b> 
        <br>- Lots of MetaData on orders. 
        <br>- User Session listed, possible multiple orders during sessions accounted for.
    </p>
    <p><b>Negatives:</b> 
        <br>- No meta data on customers. Kaggle site says 'all personal data removed'. Useless for Customer Segmentation.
        <br>- Category Code limits dataset
   
</div>

<div class="span5 alert alert-info">
<h3>Online Electronics Store Data</h3>
</div>

In [10]:
PrintDataFrameSummary(eElecData)

Name:                DType:         Non-Null:        Unique:        
event_time           object         2633521          1316174        
order_id             int64          2633521          1435266        
product_id           int64          2633521          25113          
category_id          float64        2201567          900            
category_code        object         2021319          510            
brand                object         2127516          23021          
price                float64        2201567          2802           
user_id              float64        564169           98262          

Initial Length 2633521
Dropped NA Length 420718


,event_time,order_id,product_id,category_id,category_code,brand,price,user_id
0,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
1,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
2,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
3,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
4,2020-04-24 19:16:21 UTC,2294584263154074236,2273948316817424439,2.268105e+18,NaN,karcher,217.57,1.515916e+18


<div class="span5 alert alert-info">
    <p><b>Positives:</b> 
        <br>- Lots of MetaData on orders. 
        <br>- User Session listed, possible multiple orders during sessions accounted for.
    </p>
    <p><b>Negatives:</b> 
        <br>- No meta data on customers. Useless for Customer Segmentation.
        <br>- User ID limits Data set, and other null paramters limit it more (564k -> 420k)
</div>

<div class="span5 alert alert-info">
<h3>IBM Example Telco Customer Data</h3>
</div>

In [11]:
PrintDataFrameSummary(ibmTelcoData)

Name:                   DType:         Non-Null:        Unique:        
customerID              object         7043             7043           
gender                  object         7043             2              
SeniorCitizen           int64          7043             2              
Partner                 object         7043             2              
Dependents              object         7043             2              
tenure                  int64          7043             73             
PhoneService            object         7043             2              
MultipleLines           object         7043             3              
InternetService         object         7043             3              
OnlineSecurity          object         7043             3              
OnlineBackup            object         7043             3              
DeviceProtection        object         7043             3              
TechSupport             object         7043             3       

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


<div class="span5 alert alert-info">
    <p><b>Positives:</b> 
        <br>- Many MetaData features on Users
        <br>- Many MetaData features on Service Provided  
        <br>- Churn given as parameter, easy to make dependent variable
    </p>
    <p><b>Negatives:</b> 
        <br>- Smaller Data Set
        <br>- No Date data in set.  
        <br>- Generated Data from IBM, not real world data
</div>

<div class="span5 alert alert-info">
<h3>IBM Example Marketing Data</h3>
</div>

In [12]:
PrintDataFrameSummary(ibmMarketingData)

Name:                                DType:         Non-Null:        Unique:        
Customer                             object         9134             9134           
State                                object         9134             5              
Customer Lifetime Value              float64        9134             8041           
Response                             object         9134             2              
Coverage                             object         9134             3              
Education                            object         9134             5              
Effective To Date                    object         9134             59             
EmploymentStatus                     object         9134             5              
Gender                               object         9134             2              
Income                               int64          9134             5694           
Location Code                        object         9134         

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


<div class="span5 alert alert-info">
    <p><b>Positives:</b> 
        <br>- No missing Data
        <br>- Many MetaData features on Users
        <br>- Many MetaData features on Service Provided  
        <br>- Customer easy to make dependent variable
    </p>
    <p><b>Negatives:</b> 
        <br>- Smaller Data Set
        <br>- No Date data in set.  
        <br>- Generated Data from IBM, not real world data
</div>

<div class="span5 alert alert-info">
<h3>Mall Customers</h3>
</div>

In [13]:
PrintDataFrameSummary(mallData)

Name:                         DType:        Non-Null:        Unique:        
CustomerID                    int64         200              200            
Gender                        object        200              2              
Age                           int64         200              51             
Annual Income (k$)            int64         200              64             
Spending Score (1-100)        int64         200              84             

Initial Length 200
Dropped NA Length 200


,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


<div class="span5 alert alert-info">
    <p><b>Positives:</b> 
        <br>- Some MetaData features on Users 
        <br>- Spending Score, easy to make dependent variable
    </p>
    <p><b>Negatives:</b> 
        <br>- Small Data Set
        <br>- No Date data in set
        <br>- No data on services provided
        <br>- No calculation given for Spending Score, nor parameters to calculate our own
</div>

<div class="span5 alert alert-info">
<h3>UCI Online Retail 1</h3>
</div>

In [14]:
PrintDataFrameSummary(uciOnlineRetail1)

Name:              DType:         Non-Null:        Unique:        
InvoiceNo          object         541909           25900          
StockCode          object         541909           4070           
Description        object         540455           4223           
Quantity           int64          541909           722            
InvoiceDate        object         541909           23260          
UnitPrice          float64        541909           1630           
CustomerID         float64        406829           4372           
Country            object         541909           38             

Initial Length 541909
Dropped NA Length 406829


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom


<div class="span5 alert alert-info">
    <p><b>Positives:</b> 
        <br>- Some MetaData on order location
        <br>- Dates provided
        <br>- Some MetaData on orders
    </p>
    <p><b>Negatives:</b> 
        <br>- No Meta Data on users
</div>

<div class="span5 alert alert-info">
<h3>UCI Online Retail 2</h3>
</div>

In [15]:
PrintDataFrameSummary(uciOnlineRetail2)

Name:              DType:         Non-Null:        Unique:        
Invoice            object         525461           28816          
StockCode          object         525461           4632           
Description        object         522533           4681           
Quantity           int64          525461           825            
InvoiceDate        object         525461           25296          
Price              float64        525461           1606           
Customer ID        float64        417534           4383           
Country            object         525461           40             

Initial Length 525461
Dropped NA Length 417534


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/1/09 7:45,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,12/1/09 7:45,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/1/09 7:45,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/1/09 7:45,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/1/09 7:45,1.25,13085.0,United Kingdom


<div class="span5 alert alert-info">
    <p><b>Positives:</b> 
        <br>- Some MetaData on order location
        <br>- Dates provided
        <br>- Some MetaData on orders
    </p>
    <p><b>Negatives:</b> 
        <br>- No Meta Data on users
</div>

<div class="span5 alert alert-info">
<h3>ULA Box Orders</h3>
</div>

In [16]:
PrintDataFrameSummary(ulaboxOrders)

Name:              DType:         Non-Null:        Unique:        
customer           int64          30000            10239          
order              int64          30000            30000          
total_items        int64          30000            173            
discount%          float64        30000            3579           
weekday            int64          30000            7              
hour               int64          30000            24             
Food%              float64        30000            6344           
Fresh%             float64        30000            7002           
Drinks%            float64        30000            6714           
Home%              float64        30000            5412           
Beauty%            float64        30000            3160           
Health%            float64        30000            1689           
Baby%              float64        30000            4677           
Pets%              float64        30000            1394       

,customer,order,total_items,discount%,weekday,hour,Food%,Fresh%,Drinks%,Home%,Beauty%,Health%,Baby%,Pets%
0,0,0,45,23.03,4,13,9.46,87.06,3.48,0.00,0.00,0.00,0.0,0.0
1,0,1,38,1.22,5,13,15.87,75.80,6.22,2.12,0.00,0.00,0.0,0.0
2,0,2,51,18.08,4,13,16.88,56.75,3.37,16.48,6.53,0.00,0.0,0.0
3,1,3,57,16.51,1,12,28.81,35.99,11.78,4.62,2.87,15.92,0.0,0.0
4,1,4,53,18.31,2,11,24.13,60.38,7.78,7.72,0.00,0.00,0.0,0.0


<div class="span5 alert alert-info">
    <p><b>Positives:</b> 
        <br>- No nulls in Dataset
        <br>- Department breakdown of Orders
        <br>- Some MetaData on orders
    </p>
    <p><b>Negatives:</b> 
        <br>- No metadata on customers
        <br>- Weekday and hour given, but not date
        <br>- customer and order numbering make it seem like this data was generated, unrealistic
</div>

<div class="span5 alert alert-info">
<h3>OList eCommerce Public Data Set</h3>
</div>

In [17]:
PrintDataFrameSummary(OL_Data)

Name:                                DType:         Non-Null:        Unique:        
order_id                             object         116554           97659          
customer_id                          object         116554           97659          
order_status                         object         116554           3              
order_purchase_timestamp             object         116554           97115          
order_approved_at                    object         116539           89287          
order_delivered_carrier_date         object         116553           81018          
order_delivered_customer_date        object         115228           95664          
order_estimated_delivery_date        object         116554           446            
payment_sequential                   float64        116551           29             
payment_type                         object         116551           4              
payment_installments                 float64        116551       

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,...,seller_city,seller_state,seller_latitude,seller_longitude,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,customer_latitude,customer_longitude
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,credit_card,...,maua,SP,-23.680114,-46.452454,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,-23.574809,-46.587471
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,3.0,voucher,...,maua,SP,-23.680114,-46.452454,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,-23.574809,-46.587471
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,2.0,voucher,...,maua,SP,-23.680114,-46.452454,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,-23.574809,-46.587471
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1.0,boleto,...,belo horizonte,SP,-19.810119,-43.984727,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,-12.169860,-44.988369
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1.0,credit_card,...,guariba,SP,-21.362358,-48.232976,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,-16.746337,-48.514624


<div class="span5 alert alert-info">
    <p><b>Positives:</b> 
        <br>- Many metadata features on Customers
        <br>- Many metadata features on Orders
        <br>- Many metadata features on Sellers
        <br>- Many metadata features on Products
        <br>- Time Data on Purchase and Delivery data
        <br>- Real Data from an actual seller
    </p>
    <p><b>Negatives:</b> 
        <br>- Lots of datasets in separate files, lots of validation needed
         
</div>

<div class="span5 alert alert-success">
    <h3>Part 1 Conclusions</h3>    
    <br>- Clear winner in terms of data for cutomer segmentation is OList Data set
    <br>- IBM sets are a close second, with generated, but representative Data
</div>  

<h2>Part 2) Data Validation</h2>

<h4>dtypes</h4>

In [18]:
OL_Data.dtypes

order_id                          object
customer_id                       object
order_status                      object
order_purchase_timestamp          object
order_approved_at                 object
order_delivered_carrier_date      object
order_delivered_customer_date     object
order_estimated_delivery_date     object
payment_sequential               float64
payment_type                      object
payment_installments             float64
payment_value                    float64
review_id                         object
review_score                     float64
review_creation_date              object
review_answer_timestamp           object
order_item_id                    float64
product_id                        object
seller_id                         object
shipping_limit_date               object
price                            float64
freight_value                    float64
product_weight_g                 float64
product_length_cm                float64
product_height_c

<h4>Convert Timestamps</h4>

In [19]:
for dateCol in ['order_purchase_timestamp',
                'order_approved_at',
                'order_delivered_carrier_date',
                'order_delivered_customer_date', 
                'order_estimated_delivery_date',
                'review_creation_date',
                'review_answer_timestamp',
                'shipping_limit_date'
               ]:
    OL_Data[dateCol] = pd.to_datetime(OL_Data[dateCol])

<h4>Categorical Data Investigation</h4>

In [20]:
OL_Data['order_status'].value_counts()

delivered    115228
shipped        1246
canceled         80
Name: order_status, dtype: int64

In [21]:
OL_Data['payment_type'].value_counts()

credit_card    86005
boleto         22629
voucher         6237
debit_card      1680
Name: payment_type, dtype: int64

<h4>Numeric Data Investigation</h4>

In [22]:
OL_Data['payment_sequential'].value_counts()

1.0     111542
2.0       3344
3.0        641
4.0        311
5.0        186
6.0        129
7.0         88
8.0         57
9.0         47
10.0        40
11.0        36
12.0        28
13.0        16
14.0        13
15.0        11
16.0         9
18.0         9
17.0         9
19.0         9
21.0         6
20.0         6
22.0         3
25.0         2
26.0         2
23.0         2
24.0         2
27.0         1
29.0         1
28.0         1
Name: payment_sequential, dtype: int64

In [23]:
OL_Data['payment_installments'].value_counts()

1.0     58082
2.0     13617
3.0     11653
4.0      7916
10.0     6771
5.0      5962
8.0      5008
6.0      4590
7.0      1806
9.0       717
12.0      164
15.0       91
18.0       38
24.0       34
11.0       25
20.0       21
13.0       17
14.0       15
17.0        7
16.0        7
21.0        5
0.0         3
23.0        1
22.0        1
Name: payment_installments, dtype: int64

In [24]:
OL_Data['payment_value'].value_counts()

50.00     349
100.00    295
20.00     284
77.57     250
35.00     162
         ... 
255.70      1
200.88      1
286.34      1
166.56      1
7.34        1
Name: payment_value, Length: 28707, dtype: int64

In [25]:
print("{0} null".format(sum(OL_Data['review_score'].isna())))
OL_Data['review_score'].value_counts()

65753 null


5.0    25000
1.0    10967
4.0     7422
3.0     4567
2.0     2845
Name: review_score, dtype: int64

In [26]:
OL_Data['order_item_id'].value_counts()

1.0     102161
2.0      10159
3.0       2346
4.0        962
5.0        453
6.0        257
7.0         60
8.0         36
9.0         28
10.0        25
11.0        17
12.0        13
13.0         8
14.0         7
15.0         5
16.0         3
17.0         3
18.0         3
19.0         3
20.0         3
21.0         1
Name: order_item_id, dtype: int64

In [27]:
print(OL_Data['price'].describe())

count    116553.000000
mean        120.178269
std         182.908269
min           0.850000
25%          39.900000
50%          74.900000
75%         133.990000
max        6735.000000
Name: price, dtype: float64


In [28]:
print(OL_Data['freight_value'].describe())

count    116553.000000
mean         20.024776
std          15.782010
min           0.000000
25%          13.080000
50%          16.310000
75%          21.190000
max         409.680000
Name: freight_value, dtype: float64


In [29]:
print(OL_Data['product_weight_g'].describe())

count    114887.000000
mean       2116.251264
std        3779.557862
min           0.000000
25%         300.000000
50%         700.000000
75%        1800.000000
max       40425.000000
Name: product_weight_g, dtype: float64


In [30]:
print(OL_Data['product_length_cm'].describe())
print(OL_Data['product_height_cm'].describe())
print(OL_Data['product_width_cm'].describe())

count    114887.000000
mean         30.306379
std          16.203706
min           7.000000
25%          18.000000
50%          25.000000
75%          38.000000
max         105.000000
Name: product_length_cm, dtype: float64
count    114887.000000
mean         16.620914
std          13.446639
min           2.000000
25%           8.000000
50%          13.000000
75%          20.000000
max         105.000000
Name: product_height_cm, dtype: float64
count    114887.000000
mean         23.103920
std          11.742616
min           6.000000
25%          15.000000
50%          20.000000
75%          30.000000
max         118.000000
Name: product_width_cm, dtype: float64


In [31]:
print(OL_Data['product_volume_cm3'].describe())
print(OL_Data['product_density_g_per_cm3'].describe())

count    114887.000000
mean      15364.972234
std       23533.766231
min         168.000000
25%        2856.000000
50%        6650.000000
75%       18600.000000
max      296208.000000
Name: product_volume_cm3, dtype: float64
count    114887.000000
mean          0.191743
std           0.789217
min           0.000000
25%           0.067268
50%           0.112500
75%           0.191474
max          85.227273
Name: product_density_g_per_cm3, dtype: float64


<h4>Location Data Investigation</h4>

In [32]:
print(OL_Data['seller_zip_code_prefix'])
print(OL_Data['customer_zip_code_prefix'])

0          9350.0
1          9350.0
2          9350.0
3         31570.0
4         14840.0
           ...   
116549    17602.0
116550     8290.0
116551    37175.0
116552    37175.0
116553    14407.0
Name: seller_zip_code_prefix, Length: 116554, dtype: float64
0          3149
1          3149
2          3149
3         47813
4         75265
          ...  
116549    11722
116550    45920
116551    28685
116552    28685
116553    83750
Name: customer_zip_code_prefix, Length: 116554, dtype: int64


<h4>Writing out Data</h4>

In [33]:
OL_Data.dtypes

order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
payment_sequential                      float64
payment_type                             object
payment_installments                    float64
payment_value                           float64
review_id                                object
review_score                            float64
review_creation_date             datetime64[ns]
review_answer_timestamp          datetime64[ns]
order_item_id                           float64
product_id                               object
seller_id                                object
shipping_limit_date              datetime64[ns]
price                                   

In [34]:
OL_Data.to_csv("./OL_Data.csv")

<h3>Generate Columns</h3>

In [35]:
#Total Price and Total Items in Order
itemSub = OL_Data[['order_id', 'order_item_id','price','product_id']]
itemSub = itemSub.drop_duplicates()
itemGroup = itemSub.groupby('order_id', as_index=False).sum()
itemGroup['number_of_items'] = itemGroup['order_item_id'].map({0:0,
                                                                1:1,
                                                                3:2,
                                                                6:3,
                                                                10:4,
                                                                15:5,
                                                                21:6,
                                                                28:7,
                                                                36:8,
                                                                45:9,
                                                                55:10,
                                                                66:11,
                                                                78:12,
                                                                91:13,
                                                                105:14,
                                                                120:15,
                                                                210:20,
                                                                231:21})
itemGroup = itemGroup.drop(['order_item_id'],axis=1)
itemGroup = itemGroup.rename(columns={"price":"Total_price"})
print("itemGroup")
print(itemGroup.head())


#Median Payments, useful for installment plans
paymentSub = OL_Data[['order_id',
                      'payment_value',
                      'payment_type',
                      'freight_value',
                     'payment_sequential']]
paymentSub = paymentSub.drop_duplicates()
payments = paymentSub[['order_id','payment_value','freight_value']]
paymentsMed = payments[['order_id','payment_value']].groupby('order_id', as_index=False).median()
paymentsMed = paymentsMed.rename(columns ={"payment_value":"Median_payments"})
print()
print("paymentsMed")
print(paymentsMed.head())

#Total Payments, Total Frieght Cost Value
paymentsTot = payments.groupby('order_id', as_index=False).sum()
paymentsTot = paymentsTot.rename(columns = {"payment_value":"Total_payment","freight_value":"Total_Freight"})
print()
print("paymentsTot")
print(paymentsTot.head())

#Total Weight and Volume of Order
ordersSize = OL_Data[['order_id', 
                        'order_item_id',
                        'price',
                        'product_id',
                        'product_weight_g',
                        'product_volume_cm3']]
ordersSize = ordersSize.drop_duplicates()
ordersSize = ordersSize[['order_id',
                         'product_weight_g',
                        'product_volume_cm3']].groupby('order_id', as_index=False).sum()
ordersSize = ordersSize.rename(columns = {'product_weight_g':'order_weight_g',
                                         'product_volume_cm3':'order_volume_cm3'})
print()
print("ordersSize")
print(ordersSize.head())

#Payment Type
paymentsType = paymentSub[['order_id','payment_sequential','payment_type']]
paymentsType = paymentsType.sort_values(by='payment_sequential')[['order_id', 'payment_type']]
paymentsType = paymentsType.groupby('order_id', as_index=False).first()
print()
print("paymentsType")
print(paymentsType.head())

itemGroup
                           order_id  Total_price  number_of_items
0  00010242fe8c5a6d1ba2dd792cb16214        58.90                1
1  00018f77f2f0320c557190d7a144bdd3       239.90                1
2  000229ec398224ef6ca0657da4fc703e       199.00                1
3  00024acbcdf0a6daa1e931b038114c75        12.99                1
4  00042b26cf59d7ce69dfabb4e55b4fd9       199.90                1

paymentsMed
                           order_id  Median_payments
0  00010242fe8c5a6d1ba2dd792cb16214            72.19
1  00018f77f2f0320c557190d7a144bdd3           259.83
2  000229ec398224ef6ca0657da4fc703e           216.87
3  00024acbcdf0a6daa1e931b038114c75            25.78
4  00042b26cf59d7ce69dfabb4e55b4fd9           218.04

paymentsTot
                           order_id  Total_payment  Total_Freight
0  00010242fe8c5a6d1ba2dd792cb16214          72.19          13.29
1  00018f77f2f0320c557190d7a144bdd3         259.83          19.93
2  000229ec398224ef6ca0657da4fc703e         216.87  

In [36]:
#Merge to Single DataFrame.
#An order may still have multiple sellers
NonGroupedData = OL_Data[['order_id',  
                          'order_status', 
                          'order_purchase_timestamp',
                          'order_approved_at', 
                          'order_delivered_carrier_date',
                          'order_delivered_customer_date', 
                          'order_estimated_delivery_date',
                          'shipping_limit_date',
                          'payment_installments',
                          'product_category_name_english', 
                          'seller_id', 
                          'seller_zip_code_prefix',
                          'seller_city', 
                          'seller_state', 
                          'seller_latitude', 
                          'seller_longitude',
                          'customer_id',
                          'customer_unique_id', 
                          'customer_zip_code_prefix', 
                          'customer_city',
                          'customer_state', 
                          'customer_latitude', 
                          'customer_longitude',
                          'review_id', 
                          'review_score', 
                          'review_creation_date',
                          'review_answer_timestamp']]
NonGroupedData= NonGroupedData.drop_duplicates()
OrderData = NonGroupedData.merge(itemGroup,on='order_id')
OrderData = OrderData.merge(paymentsMed, on='order_id')
OrderData = OrderData.merge(paymentsTot, on='order_id')
OrderData = OrderData.merge(ordersSize, on='order_id')
OrderData = OrderData.merge(paymentsType, on='order_id')

In [37]:
#Generate Delivery Features
OrderData['Purchase_To_Delivery_Days'] = [x.days for x in OrderData['order_delivered_customer_date']-OrderData['order_purchase_timestamp']]
OrderData['Approved_To_Delivery_Days'] = [x.days for x in OrderData['order_delivered_customer_date']-OrderData['order_approved_at']]
OrderData['Diff_Est_Delivery_vs_Actual'] = [x.days for x in OrderData['order_delivered_customer_date']-OrderData['order_estimated_delivery_date']]

<h4>Write Out Data</h4>

In [38]:
OrderData.to_csv("./OL_Data_Order_Grouped.csv")